In [1]:
# 1. 패키지 설치 및 불러오기
install.packages("mirt")  # 최초 1회만
library(mirt)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: stats4

Loading required package: lattice



In [2]:

# 2. 데이터 불러오기
data <- read.csv("/processed_df_revision.csv")  # 열 이름: EXT1~EXT10, EST1~EST10, 등

In [3]:
model_spec <- '
EXT = 1-10
EST = 11-20
AGR = 21-30
CSN = 31-40
OPN = 41-50
'


In [5]:
# 예: 1000명만 샘플링
set.seed(42)
sub_data <- data[sample(1:nrow(data), 1000), ]
mod <- mirt(sub_data, model_spec, itemtype = "graded")

EM quadrature for high dimensional models are better handled
                                 	with the "QMCEM" or "MCEM" method



Iteration: 48, Log-Lik: -65906.246, Max-Change: 0.00008


In [16]:
# θ = 0 지점에서 각 문항의 정보량 추출
theta_0 <- matrix(0, ncol = 5)  # 다차원 모델이므로 θ 벡터 필요
info_values <- numeric(50)


for (i in 1:50) {
  info_values[i] <- testinfo(mod, Theta = theta_0, which.items = i, degrees = c(0, 0, 0, 0, 0))
}

# 결과 저장
info_df <- data.frame(
  Item = colnames(sub_data),
  Info_theta0 = info_values
)

In [17]:
# 문항별 파라미터 추출 (변별도 a1~a5 포함)
param <- coef(mod, IRTpars = TRUE, simplify = TRUE)$items

# 변별도 추출 (a1~a5), 문항 이름 매칭
a_params <- param[, grep("^a", colnames(param))]
colnames(a_params) <- paste0("a", 1:5)
a_df <- as.data.frame(a_params)
a_df$Item <- rownames(a_params)

In [18]:
# 두 데이터프레임 병합
merged_df <- merge(info_df, a_df, by = "Item")

# 저장
write.csv(merged_df, "/item_info_with_discrimination.csv", row.names = FALSE)

In [21]:
write.csv(merged_df, "/item_info_with_discrimination.csv", row.names = FALSE)